# MANUela Anomaly ML Model

(original source available at: https://github.com/sa-mw-dach/manuela-dev/blob/master/ml-models/anomaly-detection/Anomaly-Detection-simple-ML-Training.ipynb)

Goal: Build a machine lerning model that detects anomalies in sensor vibration data

![anomalies](https://raw.githubusercontent.com/sa-mw-dach/manuela/master/docs/images/manuela-anomalies.png)

**Outline:**

### Read and preview the labeled data
- Read data from disk
- Preview the raw data
- Visualize vibration and anomalies

### Data wrangling
- Convert time series data into small episodes that can be used for supervised learning.
- Explore the new data format

### Model training
- Prepare the data for modeling, training and testing
- Train and validate model

### Save the new model
- Save model to disk
- Commit and push to git

###  Model serving during runtime
- View the warpper code for the model serving with Seldon



## Read data from disk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('raw-data.csv')
df['time'] = pd.to_datetime(df['ts'],unit='ms')
df.set_index('time', inplace=True)
df.drop(columns=['ts'], inplace=True)

### Preview the raw data
Quick view on the data. It is a dataframe with a 
- timeseries, 
- an id for the pump, 
- the vibration value
- and a label indicating an anmonaly.

In [ ]:
df.tail(5)

### Visualize data
- The uppper graph show a subset of the vibration data.
- The lower graph illustrates the anomalies (1 = anomaly, 0 = normal)

In [ ]:
df21 = df.head(100)
df21.plot(figsize=(20,10), fontsize=12,subplots=True, style=["-","o"], title = "Pump 2 - 100 Values")
plt.show()

## Data Wrangling
Goal: Convert time series data into small episodes that can be uses for supervised learning.

In [ ]:
#
# Few helper functions
#

# Get list with column names: F1, F2, Fn, L
def get_columns(n):
    f = []
    for x in range(1,n+1):
        f.append("F"+str(x))
    f.append("L")
    return f

# Create empty data frame
def create_empty_df(n):
    d= ([0.]*n)
    d.append(0)
    dfx = pd.DataFrame([d], columns=get_columns(n))
    dfx.drop(dfx.index[0], inplace=True)
    return dfx

# Create data frame with one row
def create_df(vals: list, label: int = 0):
    if not isinstance(vals, list):
        raise TypeError
    #vals.append(label)    
    dfx = pd.DataFrame([vals+[label]], columns=get_columns(len(vals)))
    return dfx




Create a new dataframe: Rows represent the last x (length) value and the label.


```
| tz | value | label |
|----|-------|-------|
| .. |   ... |   .   |
| 04 |   6.2 |   0   |
| 05 |   7.2 |   0   |
| 06 |   5.1 |   0   |
| 07 |   6.4 |   0   |
| 08 |   2.2 |   0   |
| 09 |  12.4 |   0   |
| 10 |   8.4 |   1   |
| .. |   ... |   .   |
```

Convert to episodes with lenght = 5

```

| F1 | F2 | F3 | F4 | F5 |  L  |
|----|----|----|----|----|---- |
| 6.2| 7.2| 5.1| 6.4| 2.2|  0  |
| 7.2| 5.1| 6.4| 2.2|12.4|  1  |
| 5.1| 6.4| 2.2|12.4| 8.4|  0  |


```



In [ ]:
length = 5 # Episode lenght

df_epis = create_empty_df(length)

for id in df.id.unique():
    print("Convert data for: ", id)
    
    df2 = df.loc[df['id'] == id]

    epi = []
    for index, row in df2.iterrows():
        # print('%6.2f, %d' % (row['value'], row['label']))
        epi.append(row['value'])
        if len(epi) == length :
            df_row = create_df(epi,row['label'] )
            df_epis = df_epis.append(df_row, ignore_index=True)
            del(epi[0])

### Explore the new data format
Show episodes of the lengh 5 and the label in the last column

In [ ]:
df_epis.tail(5)

The data is ready for supervised learning

## Prepare the data for modeling, training and testing
- Identify and separate the feature and target columns
- Training and Testing Data Split

### Identify feature and target columns
Like many ML libraries, sklern requires separted feature (X) and target (Y) columns. 

In [ ]:
# Extract feature columns
feature_cols = list(df_epis.columns[:-1])

# Extract target column 'label'
target_col = df_epis.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = df_epis[feature_cols]
y_all = df_epis[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

### Training and Testing Data Split
It is best best practise to have separate sets for the training and test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X_all, y_all, test_size=0.33, random_state=42)


# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

print ("Anomaly rate of the training set: {:.2f}%".format(100 * (y_train == 1).mean()))
print ("Anomaly rate of the testing set: {:.2f}%".format(100 * (y_test == 1).mean()))

### Model Training

Let's do a quick model training with a Decision Tree Classifier.

In [ ]:
# from sklearn import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

my_random_seed = 42

# Initialize the DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=my_random_seed)

# train the classifier
model.fit(X_train, y_train)

# Predict with the test data
y_predict = model.predict(X_test)

# Determine the accuracy score
accuracy_score = accuracy_score(y_test, y_predict)

print ("Accuracy score for the trained model: {:.4f}.".format(accuracy_score))

... the accuracy is pretty good.


### Save model to disk
- Save the model
- Load the model again and check that it works

In [ ]:
from joblib import dump, load

filename = 'model.joblib'
dump(model, open(filename, 'wb'))

# Validate that the model can be loaded

# load the model from disk
loaded_model = load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print("Score:", result)

### Commit the new model to git
The new model is saved in git so that the Tekton pipeline can build a new container image with the model
```
git add model.joblib
git commit -m "Update ML model"
pit push
```

###  Model serving during runtime
Have a look here to view the warpper code for the model serving with Seldon [AnomalyDetection.py](https://github.com/sa-mw-dach/manuela-dev/blob/master/components/iot-anomaly-detection/AnomalyDetection.py#L11)